In [ ]:
import numpy as np
import os
from PIL import Image
import os
import xml.etree.ElementTree as ET
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
# function to load images from specified folder path
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):
            try:
                img = Image.open(img_path)
                images.append(img)
            except Exception as e:
                print(f"Error loading image: {img_path} - {e}")
    return images

In [ ]:
# Specify the folder path containing the images
folder_path = './images'

# Load images from the folder
images = load_images_from_folder(folder_path)

# Display the first image
if len(images) > 0:
    images[0].show()
else:
    print("No images found in the folder.")

In [ ]:
# Finds class and boundary box data in .xml files 
def parse_xml_label(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Extract image size (optional)
    size_elem = root.find('size')
    width = int(size_elem.find('width').text)
    height = int(size_elem.find('height').text)
    depth = int(size_elem.find('depth').text)
    
    # Extract object annotations
    annotations = []
    for obj_elem in root.findall('object'):
        name = obj_elem.find('name').text
        bbox_elem = obj_elem.find('bndbox')
        xmin = int(bbox_elem.find('xmin').text)
        ymin = int(bbox_elem.find('ymin').text)
        xmax = int(bbox_elem.find('xmax').text)
        ymax = int(bbox_elem.find('ymax').text)
        annotations.append({
            'name': name,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax
        })
    
    return width, height, depth, annotations

In [ ]:
# Specify the directory path containing the XML labels
labels_dir = './images'

# Load .xml labels for all files in the directory
# This is a dictionary object
labels = {}
for filename in os.listdir(labels_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(labels_dir, filename)
        width, height, depth, annotations = parse_xml_label(xml_path)
        labels[filename] = {
            'width': width,
            'height': height,
            'depth': depth,
            'annotations': annotations
        }

In [ ]:
# This is a class for the data loader for images and labels
class YourDataset(Dataset):
    def __init__(self, images, labels, target_size):
        self.images = images
        self.labels = labels
        self.target_size = target_size
        self.transform = transforms.Compose([
            transforms.Resize(target_size),  # resize the image before converting to a tensor
            transforms.ToTensor(),  # then convert to a tensor
        ])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        # Here the index is passed to a list for images but the labels are a dict so the index is 
        #concverted to a key
        label_index = index + 1
        key = f"{label_index}.xml"  
        label = labels[key]
    
        # Transforms the image into appropriate sized tensor
        image = self.transform(image)
    
        # Convert the annotations to tensors
        annotations = label.get('annotations', [])
        annotation_tensors = []
        # This dictionary is for classification of the defects
        class_to_label = {'Point': 0, 'Void': 1, 'Point Defect': 0}

        for annotation in annotations:
            class_label = class_to_label[annotation.get('name')]
            xmin = annotation.get('xmin', 0)
            ymin = annotation.get('ymin', 0)
            xmax = annotation.get('xmax', 0)
            ymax = annotation.get('ymax', 0)
            
            annotation_tensor = torch.tensor([class_label, xmin, ymin, xmax, ymax])
            annotation_tensors.append(annotation_tensor)
        
        # Pad the annotation tensors to a fixed length
        # if images have different number of annotations it causes shape error
        max_annotations = 15
        while len(annotation_tensors) < max_annotations:
            # Append padding annotation
            pad_tensor = torch.tensor([-1, -1, -1, -1, -1])  # Padding value
            annotation_tensors.append(pad_tensor)
        
        annotation_tensors = torch.stack(annotation_tensors)  
        return image, annotation_tensors

In [ ]:
# Create an instance of your Dataset
dataset = YourDataset(images, labels, (256, 256))

# Check the length of your dataset
print(f"Dataset length: {len(dataset)}")  # should be 2

# Load the data from your Dataset

data_loader = DataLoader(dataset, batch_size=1)  # batch_size can be adjusted based on your requirement

# Get an iterator from the DataLoader
data_iter = iter(data_loader)

# Get the first item
first_item = next(data_iter)

# first_item now contains the first batch of data from your data_loader.
# It's a list where the first element is the image tensor and the second element is the annotations tensor.

# If you want to access the first image and its corresponding annotations, you can do:
first_image, first_annotations = first_item

print("First Image shape: ", first_image.shape)

In [ ]:
first_image, first_annotations